In [9]:
from binance.spot import Spot
import pandas as pd

In [27]:
client = Spot(api_key='9bWvHb3DaWCq9JSp36bieIAb0yxIvf68aBb3xS5ZHOEu2GgwPKHAj7uNpErrDPgk'
              ,api_secret='cWOP89mPckrKO6ykdiEQENdQjU5aMtZnZZVeinwlXtvAsxrRb8ZSU3Xaf76ZJuk1')

ans = client.account()
print(ans.keys())

2024-05-02 21:09:33.461 UTC DEBUG binance.api: url: https://api.binance.com/api/v3/account
2024-05-02 21:09:33.468 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.binance.com:443
2024-05-02 21:09:35.182 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "GET /api/v3/account?timestamp=1714684173459&signature=8537ea7099b14ec82fc7f5a4b7ff4f67b5bba08433946d20a271e37e5e9e3127 HTTP/1.1" 200 2573
2024-05-02 21:09:35.186 UTC DEBUG binance.api: raw response from server:{"makerCommission":5,"takerCommission":6,"buyerCommission":0,"sellerCommission":0,"commissionRates":{"maker":"0.00042000","taker":"0.00054000","buyer":"0.00000000","seller":"0.00000000"},"canTrade":true,"canWithdraw":true,"canDeposit":true,"brokered":false,"requireSelfTradePrevention":false,"preventSor":false,"updateTime":1714514670404,"accountType":"SPOT","balances":[{"asset":"BTC","free":"0.00009995","locked":"0.00000000"},{"asset":"LTC","free":"0.00000000","locked":"0.00000000"},{"asset":"ET

dict_keys(['makerCommission', 'takerCommission', 'buyerCommission', 'sellerCommission', 'commissionRates', 'canTrade', 'canWithdraw', 'canDeposit', 'brokered', 'requireSelfTradePrevention', 'preventSor', 'updateTime', 'accountType', 'balances', 'permissions', 'uid'])


In [37]:
balance = pd.DataFrame(ans['balances'])

In [44]:
order_limit = {'symbol':'BTCUSDT',
               'side': 'BUY',
               'type': 'LIMIT',
               'price': '60000',
               'quantity': '0.0001',
               'timeInForce': 'GTC'}

client.new_order(**order_limit)


2024-05-03 17:11:44.754 UTC DEBUG binance.api: url: https://api.binance.com/api/v3/order
2024-05-03 17:11:44.841 UTC DEBUG urllib3.connectionpool: Resetting dropped connection: api.binance.com
2024-05-03 17:11:45.818 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "POST /api/v3/order?symbol=BTCUSDT&side=BUY&type=LIMIT&price=60000&quantity=0.0001&timeInForce=GTC&timestamp=1714756304743&signature=b9acef62247031155b6c2af3b95918a24c1bbc0c5188da57d9b281282c5e62d2 HTTP/1.1" 200 276
2024-05-03 17:11:45.830 UTC DEBUG binance.api: raw response from server:{"symbol":"BTCUSDT","orderId":27006489566,"orderListId":-1,"clientOrderId":"mXaK5Rw81qr0R1a0hf8iRH","transactTime":1714756305615,"price":"60000.00000000","origQty":"0.00010000","executedQty":"0.00000000","cummulativeQuoteQty":"0.00000000","status":"NEW","timeInForce":"GTC","type":"LIMIT","side":"BUY","workingTime":1714756305615,"fills":[],"selfTradePreventionMode":"EXPIRE_MAKER"}


{'symbol': 'BTCUSDT',
 'orderId': 27006489566,
 'orderListId': -1,
 'clientOrderId': 'mXaK5Rw81qr0R1a0hf8iRH',
 'transactTime': 1714756305615,
 'price': '60000.00000000',
 'origQty': '0.00010000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'workingTime': 1714756305615,
 'fills': [],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [2]:
import websocket, json

In [3]:
socket = 'wss://stream.binance.com:9443/ws/btcusdt@kline_1m'

In [29]:
closes, highs, lows = [],[],[]

In [30]:
def on_message(ws, message):

    #Extracting data
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    close = candle['c']
    high = candle['h']
    low = candle['l']
    #vol = candle['v']

    if is_candle_closed:
        closes.append(close)
        highs.append(high)
        lows.append(low)

def on_close(ws):
    print('Conection Closed')

ws = websocket.WebSocketApp(socket,
                            on_message=on_message,
                            on_close=on_close)

In [31]:
ws.run_forever()

True

## Data preprocessing

In [32]:
data = pd.DataFrame({'Closes':closes,'Highs':highs, 'Lows':lows}).astype(float)
data.to_csv('data.csv', index=False)

In [12]:
import numpy as np

closes = data['Closes']
y = np.zeros(len(closes))

for i in range(1, len(closes)):
    if closes[i] > closes[i-1]:
        y[i] = 1


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([('std_scaler', StandardScaler())])

X = pipeline.fit_transform(data)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

## ML algorithm

In [16]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

svm_clf = LinearSVC(dual='auto')

params = dict(C=np.linspace(0,1,10))

grid_search = GridSearchCV(svm_clf, params, cv=3, scoring='accuracy')

In [18]:
grid_search.fit(X_train, y_train)

ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=2.

In [ ]:
best_estimator = grid_search.best_estimator_
grid_search.best_score_

## Implementating

In [20]:
def make_order(order):

    

def on_message_2(ws, message):

    #Extracting data
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    close = candle['c']
    high = candle['h']
    low = candle['l']
    #vol = candle['v']

    if is_candle_closed:
        closes.append(close)
        highs.append(high)
        lows.append(low)

def on_close(ws):
    print('Conection Closed')

ws = websocket.WebSocketApp(socket,
                            on_message=on_message_2,
                            on_close=on_close)

echo.websocket.events sponsored by Lob.com
